In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
try:
    import spnspecs

    spnspecs.set_graph_specifications()
except:
    spnspecs = None

In [ ]:
figpth = "../Figures"
width = 6.8
dpi = 300

In [ ]:
def drycell_smoothing(h, e, omega):
    epsilon = 0.5 * omega
    dx = h - e
    hbar = np.zeros(h.shape, dtype=np.float)
    for idx, v in enumerate(dx):
        if v < -epsilon:
            hbar[idx] = e
        elif v < epsilon:
            hbar[idx] = 0.25 * v**2 / epsilon + 0.5 * v + 0.25 * epsilon + e
        else:
            hbar[idx] = h[idx]
    return hbar

### Calculate unsmoothed and smoothed hbar

In [ ]:
dtype = np.float
step = 0.01
h0, h1 = -2, 2
h = np.arange(h0, h1, step=step, dtype=dtype)
e = 0.0

In [ ]:
h1, h.shape

In [ ]:
hbar0 = drycell_smoothing(h, e, 0.0)

In [ ]:
hbar = drycell_smoothing(h, e, 2.0)

In [ ]:
plt.plot(h, h)
plt.plot(h, hbar0)
plt.plot(h, hbar)

### Plot data

In [ ]:
fig, axes = plt.subplots(
    nrows=1, ncols=1, tight_layout=True, figsize=(width / 2, width / 2)
)
if not isinstance(axes, list):
    axes = [axes]

letters = ["A", "B"]
for idx, ax in enumerate(axes):
    ax.set_xlim(h0, h1)
    ax.set_ylim(h0, h1)
    ax.set_xticks([-1, 0, 1])
    ax.set_xticklabels(
        [r"$e_{MAW,n} - \epsilon$", r"$e_{MAW,n}$", r"$e_{MAW,n} + \epsilon$"]
    )
    ax.set_yticks([0])
    ax.set_yticklabels([r"$e_{MAW,n}$"], rotation=90, va="center")
    if spnspecs is not None:
        spnspecs.remove_edge_ticks(ax)
        # spnspecs.heading(ax, letter=letters[idx])

ax = axes[0]
ax.set_aspect("equal", "box")
ax.axhline(0, lw=0.5, ls="-.", color="black")
ax.axvline(0, lw=0.5, ls="-.", color="black")
ax.plot(
    [-3],
    [-3],
    lw=0,
    mfc="black",
    mec="none",
    ms=4.5,
    marker=r"$\epsilon$",
    label="smothing distance",
)
ax.plot(h, hbar, lw=1.25, color="#00BFFF", label=r"$\overline{h}_n^*$ continuous")
ax.plot(
    h, hbar0, lw=1.0, color="black", ls="-", label=r"$\overline{h}_n$ discontinuous"
)
if spnspecs is not None:
    handles, labels = ax.get_legend_handles_labels()
    spnspecs.graph_legend(
        ax,
        handles=handles[::-1],
        labels=labels[::-1],
        loc="lower right",
        bbox_to_anchor=(1.00, 0.05),
        handlelength=1.5,
    )
ax.set_xlabel("Downstream head, in length units")
ax.set_ylabel("Reference elevation, in length units")

fpth = os.path.join(figpth, "MAWDischargeCorrection.pdf")
fig.savefig(fpth, dpi=dpi)